In [1]:
def _fill_where(dst, src, mask) -> None:
    """Fill dst with src where mask is True"""
    if dst.ndim == 0 and ~mask:
        dst = src
    else:
        dst[mask] = src[mask]

In [2]:
from significantdigits._significantdigits import _compute_z, Error, _get_significant_size
import numpy as np

def print_hex(x):
    if isinstance(x, np.ndarray):
        for i in x:
            print_hex(i)
    elif isinstance(x, list):
        for i in x:
            print_hex(i)
    else:
        print(x.hex())

def inexact(value):
    xi = np.random.uniform(-.5, .5)
    e_x = np.floor(np.log2(abs(value))) + 1
    y = value + 2**(e_x - 40) * xi
    # print(f"value: {value}, e_x: {e_x}, xi: {xi}, y: {y}")
    return y

def add(x,y):
    return inexact(x+y)

def sub(x,y):
    return inexact(x-y)

def mul(x,y):
    return inexact(x*y)

def div(x,y):
    return inexact(x/y)

a = np.array([[0.2161, 0.1441], [1.2969, 0.8648]])
b = np.array([0.1440, 0.8642])

def cramer(a, b):
    det = sub(mul(a[0, 0], a[1, 1]) , mul(a[1, 0],  a[0, 1]))
    det0 = sub(mul(b[0] , a[1, 1]) , mul(b[1], a[0, 1]))
    det1 = sub(mul(a[0, 0] , b[1]) , mul(a[1, 0],  b[0]))
    return np.array([div(det0 , det), div(det1, det)])

array = []
for i in range(10000):
    y= cramer(a, b)
    array.append(y)
    if i < 10:
        print([(i.hex(), f"{i:.17e}") for i in y])


array = np.array(array)
reference = np.array([2,-2])
error = Error.Absolute
dtype = np.float64
axis = 0

[('0x1.fffff92d9636ap+0', '1.99999959338148381e+00'), ('-0x1.000042de749ebp+1', '-2.00000797140204645e+00')]
[('0x1.fffed028bb419p+0', '1.99998188967152246e+00'), ('-0x1.fffd8c9ef3832p+0', '-1.99996260529179581e+00')]
[('0x1.0000ca4329aa9p+1', '2.00002411155158599e+00'), ('-0x1.0000ff9ac0f4cp+1', '-2.00003047043165871e+00')]
[('0x1.000108a3895bep+1', '2.00003154740508560e+00'), ('-0x1.00009149ca408p+1', '-2.00001731970815300e+00')]
[('0x1.ffff178506bb9p+0', '1.99998614309036626e+00'), ('-0x1.ffff418b9d86ep+0', '-1.99998864801957721e+00')]
[('0x1.ffff12fac54d7p+0', '1.99998587248164994e+00'), ('-0x1.fffea9e15aecbp+0', '-1.99997960807643227e+00')]
[('0x1.fffd40adcbacep+0', '1.99995807879502108e+00'), ('-0x1.fffc43a4bbdf7p+0', '-1.99994299671004483e+00')]
[('0x1.fffffbb1508cep+0', '1.99999974326106011e+00'), ('-0x1.0000256a7078dp+1', '-2.00000446030839507e+00')]
[('0x1.0000f0b405b06p+1', '2.00002869405887207e+00'), ('-0x1.0000ba1499536p+1', '-2.00002218251997999e+00')]
[('0x1.fffdd13717c6

## CNH

### Significant digits

In [3]:
import scipy.stats
from significantdigits._significantdigits import _internal_dtype, _default_confidence


confidence = 0.95
probability = 0.95
z, e = _compute_z(
    array, reference, error, axis=axis
)
nb_samples = z.shape[axis]
std = z.std(axis=axis, dtype=_internal_dtype)
# if std == 0, we set it to the maximum value of z 
# to avoid returning the maximum number of bits depending on the dtype
# while it can be lower (cf. Cramer example)
z_eps = np.max(np.abs(z), axis=axis)
_fill_where(z_eps, std, std == 0)
# We need to mask the std where z_eps == 0
# In that case, we have no variance and z = 0
std0 = np.ma.array(std, mask=(z_eps == 0))    
chi2 = scipy.stats.chi2.interval(confidence, nb_samples - 1)[0]
inorm = scipy.stats.norm.ppf((probability + 1) / 2)
delta_chn = 0.5 * np.log2((nb_samples - 1) / chi2) + np.log2(inorm)
significant = -np.ma.log2(std0) + (e - 1) - delta_chn
std0 = np.ma.array(std, mask=std == 0)
max_bits = _get_significant_size(z, dtype=dtype) + (e - 1)
if significant.ndim == 0:
    significant = np.ma.array(significant, mask=std0.mask)
significant = significant.filled(fill_value=max_bits - delta_chn)
significant


array([15.68105801, 15.55905898])

## General

### Significant digits

In [4]:
z, e = _compute_z(array, reference, error, axis=axis)
sample_shape = tuple(dim for i, dim in enumerate(z.shape) if i != axis)
max_bits = _get_significant_size(z, dtype=dtype)
significant = np.full(shape=sample_shape, fill_value=max_bits + (e - 1))
mask = np.full_like(significant, True, dtype=bool)
z = np.abs(z)

k_trace = []
significant_trace = []
proba_trace = []
if np.all(z <= 0):
    print(significant)
else:
    # Compute successes
    for k in range(0, max_bits + 1):
        # min(bool) <=> logical and
        kth = k + (e - 1.0)
        k_trace.append(kth[0])
        proba = np.sum(z<= 2**-kth, axis=axis)[0] / z.shape[0]
        proba_trace.append(proba)
        successes = np.min(z <= 2**-kth, axis=axis)
        mask = np.logical_and(mask, successes)
        # Update significant digits
        significant[mask] = kth[mask]
        significant_trace.append(significant[0])
    print(significant)


[14. 13.]


In [5]:
import plotly.express as px
px.scatter(
    x=k_trace, 
    y=proba_trace, 
    title="Significant digits probability"
).update_xaxes(title="k").update_yaxes(title="Probability").show()

### Contributing

In [6]:

z, e = _compute_z(
    array, reference, error, axis=axis
)
sample_shape = tuple(dim for i, dim in enumerate(z.shape) if i != axis)
max_bits = _get_significant_size(z, dtype=dtype)
contributing = np.full(shape=sample_shape, fill_value=1)
mask = np.full_like(contributing, True, dtype=bool)    
z = np.abs(z)
k_trace = []
k_trace_1 = []
proba_trace = []
proba_trace_1 = []
nsample = z.shape[axis]
print("nsample: ", nsample)
for k in range(1, max_bits + 1):
    # scale = ldexp(x,n) = x * 2^n
    # floor(scale) & 1 : returns 1 if scale is odd
    # taking the max to check if at least one result is odd
    # Get the negation to have success as boolean
    kth = k - (e - 1.0)
    #
    k_trace.append(kth[0])
    k_trace_1.append(kth[1])
    #
    kth_bit_z = np.floor(z * 2 ** kth).astype(np.int64)    
    successes = np.sum(kth_bit_z, axis=axis) == 0
    #
    proba = (nsample - np.sum(np.mod(kth_bit_z, 2), axis=axis)) / nsample
    proba_trace.append(proba[0])
    proba_trace_1.append(proba[1])
    #
    mask = np.logical_and(mask, successes)
    _fill_where(contributing, kth, mask)

print("contributing:", contributing)

nsample:  10000
contributing: [14 13]


In [7]:
px.scatter(
    x=k_trace, 
    y=proba_trace, 
    title="Contributing significant digits probability"
).update_xaxes(title="k", range=(-1,54)).update_yaxes(
    title="proba", range=(0,1.1)).add_hline(
        0.5, x0=0.02, x1=1, line=dict(color="darkred", dash="dash"), opacity=.65).show()

## Test Absolute/Relative on simple example

In [11]:
import plotly.graph_objects as go
import significantdigits as sd

confidence = 0.95
probability = 0.95
basis = 10
axis = 0
dtype = np.float64

# Define colors for CNH and General
cnh_color = "blue"
gen_color = "red"
color = {"CNH": cnh_color, "General": gen_color}

# Define marker symbols for Absolute and Relative
absolute_marker = "circle"
relative_marker = "square"
marker = {"Absolute": absolute_marker, "Relative": relative_marker}


def get_sd_cnh_abs(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the significant digits using the CNH method for absolute error."""
    return sd.significant_digits(
        array,
        reference=mean,
        error=sd.Error.Absolute,
        method=sd.Method.CNH,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )


def get_sd_gen_abs(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the significant digits using the General method for absolute error."""
    return sd.significant_digits(
        array,
        reference=mean,
        error=sd.Error.Absolute,
        method=sd.Method.General,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )


def get_sd_cnh_rel(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the significant digits using the CNH method for relative error."""
    return sd.significant_digits(
        array,
        reference=mean,
        error=sd.Error.Relative,
        method=sd.Method.CNH,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )

probability = 0.51
def get_sd_gen_rel(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the significant digits using the General method for relative error."""
    return sd.significant_digits(
        array,
        reference=mean,
        error=sd.Error.Relative,
        method=sd.Method.General,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )

def get_cd_cnh_abs(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the confidence digits using the CNH method for absolute error."""
    return sd.contributing_digits(
        array,
        reference=mean,
        error=sd.Error.Absolute,
        method=sd.Method.CNH,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )

def get_cd_gen_abs(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the confidence digits using the General method for absolute error."""
    return sd.contributing_digits(
        array,
        reference=mean,
        error=sd.Error.Absolute,
        method=sd.Method.General,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )

def get_cd_cnh_rel(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the confidence digits using the CNH method for relative error."""
    return sd.contributing_digits(
        array,
        reference=mean,
        error=sd.Error.Relative,
        method=sd.Method.CNH,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )

def get_cd_gen_rel(
    array,
    mean,
    axis=axis,
    confidence=confidence,
    probability=probability,
    dtype=dtype,
    basis=basis,
):
    """Compute the confidence digits using the General method for relative error."""
    return sd.contributing_digits(
        array,
        reference=mean,
        error=sd.Error.Relative,
        method=sd.Method.General,
        axis=axis,
        confidence=confidence,
        probability=probability,
        dtype=dtype,
        basis=basis,
    )


def add_sig_scatter(fig, x, y, method, error, dim):
    # Determine the row and column based on the name
    row = 1 if "Absolute" in error else 2
    fig.add_trace(
        go.Scatter(
            x=x,
            y=[val[dim - 1] for val in y],
            mode="lines+markers",
            name=method,
            marker=dict(symbol=marker[error], color=color[method]),
            legendgroup=f"{error}_Dim{dim+1}",
            legendgrouptitle=dict(text=f"{error} x[{dim-1}]"),
        ),
        row=row,
        col=dim,
    )


def add_ref_scatter(
    fig,
    x,
    y,
    method,
    error,
    dim,
    color="black",
):
    # Determine the row and column based on the name
    row = 1 if "Absolute" in error else 2
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y[:, dim - 1],
            mode="lines+markers",
            name=method,
            marker=dict(symbol=marker[error], color=color),
            legendgroup=f"{error}_Dim{dim+1}",
            legendgrouptitle=dict(text=f"{error} x[{dim-1}]"),
        ),
        row=row,
        col=dim,
    )

def add_parker_scatter(
    fig,
    x,
    y,
    method,
    error,
    dim,
    color="green",
    marker="diamond",
):
    # Determine the row and column based on the name
    row = 1 if "Absolute" in error else 2    
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y[:, dim - 1],
            mode="lines+markers",
            name=method,
            marker=dict(symbol=marker, color=color),
            legendgroup=f"{error}_Dim{dim+1}",
            legendgrouptitle=dict(text=f"{error} x[{dim-1}]"),
        ),
        row=row,
        col=dim,
    )

In [12]:
from plotly.subplots import make_subplots

shift = 1.0
magnitude = np.array([-9, -3, 4, 10], dtype=np.float64)

shifts = []
stds = []
means = []
s_abs_cnh_values = []
s_rel_cnh_values = []
s_abs_values = []
s_rel_values = []

for shift in np.arange(-8, 8, dtype=np.float64):
    mean = np.power(basis, magnitude)
    std = np.power(basis, magnitude - shift)
    array = np.random.normal(loc=mean, scale=std, size=(10000, magnitude.size))

    stds.append(std)
    means.append(mean)

    print("\nShift: ", shift)
    print("Type", array.dtype)
    print(array)
    print("-" * 20)

    print("Theorical Mean", mean)
    print("Empirical mean", array.mean(axis=0))
    print("Mean diff     ", mean - array.mean(axis=0))
    print("-" * 20)

    print("Theorical Std", std)
    print("Empirical std", array.std(axis=0))
    print("Std diff     ", std - array.std(axis=0))
    print("-" * 20)

    s_abs_cnh = get_sd_cnh_abs(array, mean)
    s_rel_cnh = get_sd_cnh_rel(array, mean)
    s_abs = get_sd_gen_abs(array, mean)
    s_rel = get_sd_gen_rel(array, mean)

    print("CNH s_abs: ", s_abs_cnh)
    print("CNH s_rel: ", s_rel_cnh)
    print("General s_abs: ", s_abs)
    print("General s_rel: ", s_rel)

    shifts.append(shift)
    s_abs_cnh_values.append(s_abs_cnh)
    s_rel_cnh_values.append(s_rel_cnh)
    s_abs_values.append(s_abs)
    s_rel_values.append(s_rel)

column_titles = [
    f"log{basis}(μ)={mag:.1f}, log{basis}(σ)={mag:.1f}-shift" for mag in magnitude
]

# Plot the results
fig = make_subplots(
    rows=2,
    cols=magnitude.size,
    column_titles=column_titles,
    row_titles=["Absolute", "Relative"],
    x_title="Shift",
    y_title="Significant Digits",
    shared_xaxes=True,
    shared_yaxes=True,
    specs=[
        [{"type": "scatter"}] * magnitude.size,
        [{"type": "scatter"}] * magnitude.size,
    ],
)


for dim, _ in enumerate(magnitude, start=1):

    add_sig_scatter(fig, shifts, s_abs_cnh_values, "CNH", "Absolute", dim)
    add_sig_scatter(fig, shifts, s_rel_cnh_values, "CNH", "Relative", dim)
    add_sig_scatter(fig, shifts, s_abs_values, "General", "Absolute", dim)
    add_sig_scatter(fig, shifts, s_rel_values, "General", "Relative", dim)


# Group Absolute and Relative traces together
fig.data = sorted(fig.data, key=lambda trace: "Absolute" in trace.name)

means = np.array(means)
stds = np.array(stds)

if basis == 2:
    e_ref = np.floor(np.log2(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log2(stds) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log2(stds / np.abs(means))
elif basis == 10:
    e_ref = np.floor(np.log10(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log10(stds) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log10(stds / np.abs(means))
else:
    e_ref = np.floor(np.log(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log(np.array(stds)) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log(np.array(stds) / np.abs(np.array(means)))
    # Convert to base basis
    abs_ref_line = abs_ref_line / np.log(basis)
    rel_ref_line = rel_ref_line / np.log(basis)


print("e_ref: ", e_ref)
print("abs_ref_line: ", abs_ref_line)
print("rel_ref_line: ", rel_ref_line)


for dim, _ in enumerate(magnitude, start=1):

    add_ref_scatter(fig, shifts, abs_ref_line, "Reference", "Absolute", dim)
    add_ref_scatter(fig, shifts, rel_ref_line, "Reference", "Relative", dim)


# Add significant digits with Parker method
if basis == 2:
    parker_sd = -np.log2(stds / np.abs(means))
elif basis == 10:
    parker_sd = -np.log10(stds / np.abs(means))
else:
    # Reference line for absolute
    parker_sd = -np.log(stds / np.abs(means))
    # Convert to base basis
    parker_sd = parker_sd / np.log(basis)


for dim, _ in enumerate(magnitude, start=1):
    add_parker_scatter(fig, shifts, parker_sd, "Parker", "Absolute", dim)
    add_parker_scatter(fig, shifts, parker_sd, "Parker", "Relative", dim)


fig.update_traces(marker=dict(size=12))
# increase the height of the figure
fig.update_layout(height=1200)

fig.show()


Shift:  -8.0
Type float64
[[-3.16113101e-02  5.74298284e+04 -4.67382081e+11  1.21924034e+18]
 [-9.31988463e-02 -1.62844554e+05 -5.73947566e+11  1.25026613e+17]
 [ 4.86059434e-02  1.22939682e+05  5.81403957e+10 -8.62946831e+16]
 ...
 [-1.82401050e-01  1.63217802e+04  2.60592089e+11 -1.21694389e+18]
 [-8.77657366e-02  2.81853105e+04 -1.50860630e+12  2.38296359e+17]
 [ 2.15839485e-02 -1.01066148e+05 -4.39694032e+11 -9.06428676e+17]]
--------------------
Theorical Mean [1.e-09 1.e-03 1.e+04 1.e+10]
Empirical mean [ 2.72817073e-04 -3.73336880e+02 -3.63380046e+09  2.57874551e+15]
Mean diff      [-2.72816073e-04  3.73337880e+02  3.63381046e+09 -2.57873551e+15]
--------------------
Theorical Std [1.e-01 1.e+05 1.e+12 1.e+18]
Empirical std [9.92687812e-02 1.00121959e+05 9.97392106e+11 9.91419161e+17]
Std diff      [ 7.31218843e-04 -1.21959269e+02  2.60789413e+09  8.58083904e+15]
--------------------
CNH s_abs:  [-8.32602244 -8.30913918 -8.38378586 -8.36057733]
CNH s_rel:  [-8.29512257 -8.29883

In [13]:
from plotly.subplots import make_subplots

shift = 1.0
magnitude = np.array([-9, -3, 4, 10], dtype=np.float64)

shifts = []
stds = []
means = []
c_abs_cnh_values = []
c_rel_cnh_values = []
c_abs_values = []
c_rel_values = []

for shift in np.arange(-8, 8, dtype=np.float64):
    mean = np.power(basis, magnitude)
    std = np.power(basis, magnitude - shift)
    array = np.random.normal(loc=mean, scale=std, size=(10000, magnitude.size))

    stds.append(std)
    means.append(mean)

    print("\nShift: ", shift)
    print("Type", array.dtype)
    print(array)
    print("-" * 20)

    print("Theorical Mean", mean)
    print("Empirical mean", array.mean(axis=0))
    print("Mean diff     ", mean - array.mean(axis=0))
    print("-" * 20)

    print("Theorical Std", std)
    print("Empirical std", array.std(axis=0))
    print("Std diff     ", std - array.std(axis=0))
    print("-" * 20)

    c_abs_cnh = get_cd_cnh_abs(array, mean)
    c_rel_cnh = get_cd_cnh_rel(array, mean)
    c_abs = get_cd_gen_abs(array, mean)
    c_rel = get_cd_gen_rel(array, mean)

    print("CNH s_abs: ", c_abs_cnh)
    print("CNH s_rel: ", c_rel_cnh)
    print("General s_abs: ", c_abs)
    print("General s_rel: ", c_rel)

    shifts.append(shift)
    c_abs_cnh_values.append(c_abs_cnh)
    c_rel_cnh_values.append(c_rel_cnh)
    c_abs_values.append(c_abs)
    c_rel_values.append(c_rel)

column_titles = [
    f"log{basis}(μ)={mag:.1f}, log{basis}(σ)={mag:.1f}-shift" for mag in magnitude
]

# Plot the results
fig = make_subplots(
    rows=2,
    cols=magnitude.size,
    column_titles=column_titles,
    row_titles=["Absolute", "Relative"],
    x_title="Shift",
    y_title="Significant Digits",
    shared_xaxes=True,
    shared_yaxes=True,
    specs=[
        [{"type": "scatter"}] * magnitude.size,
        [{"type": "scatter"}] * magnitude.size,
    ],
)


for dim, _ in enumerate(magnitude, start=1):

    add_sig_scatter(fig, shifts, c_abs_cnh_values, "CNH", "Absolute", dim)
    add_sig_scatter(fig, shifts, c_rel_cnh_values, "CNH", "Relative", dim)
    add_sig_scatter(fig, shifts, c_abs_values, "General", "Absolute", dim)
    add_sig_scatter(fig, shifts, c_rel_values, "General", "Relative", dim)


# Group Absolute and Relative traces together
fig.data = sorted(fig.data, key=lambda trace: "Absolute" in trace.name)

means = np.array(means)
stds = np.array(stds)

if basis == 2:
    e_ref = np.floor(np.log2(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log2(stds) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log2(stds / np.abs(means))
elif basis == 10:
    e_ref = np.floor(np.log10(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log10(stds) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log10(stds / np.abs(means))
else:
    e_ref = np.floor(np.log(np.abs(means))) + 1
    # Reference line for absolute
    abs_ref_line = -np.log(np.array(stds)) + (e_ref - 1)
    # Reference line for relative
    rel_ref_line = -np.log(np.array(stds) / np.abs(np.array(means)))
    # Convert to base basis
    abs_ref_line = abs_ref_line / np.log(basis)
    rel_ref_line = rel_ref_line / np.log(basis)


print("e_ref: ", e_ref)
print("abs_ref_line: ", abs_ref_line)
print("rel_ref_line: ", rel_ref_line)


for dim, _ in enumerate(magnitude, start=1):

    add_ref_scatter(fig, shifts, abs_ref_line, "Reference", "Absolute", dim)
    add_ref_scatter(fig, shifts, rel_ref_line, "Reference", "Relative", dim)


# Add significant digits with Parker method
if basis == 2:
    parker_sd = -np.log2(stds / np.abs(means))
elif basis == 10:
    parker_sd = -np.log10(stds / np.abs(means))
else:
    # Reference line for absolute
    parker_sd = -np.log(stds / np.abs(means))
    # Convert to base basis
    parker_sd = parker_sd / np.log(basis)


for dim, _ in enumerate(magnitude, start=1):
    add_parker_scatter(fig, shifts, parker_sd, "Parker", "Absolute", dim)
    add_parker_scatter(fig, shifts, parker_sd, "Parker", "Relative", dim)


fig.update_traces(marker=dict(size=12))
# increase the height of the figure
fig.update_layout(height=1200)

fig.show()


Shift:  -8.0
Type float64
[[ 5.10481070e-02 -1.98864890e+05 -1.82277975e+12 -3.74025671e+17]
 [-1.33591039e-01 -9.15490856e+04  2.22559450e+12 -1.01141636e+18]
 [ 3.97395914e-02 -7.75776171e+04 -3.38077711e+10  3.23291130e+17]
 ...
 [ 2.70793957e-03 -1.21427335e+05 -2.88798050e+11  2.45584106e+18]
 [ 1.31809687e-01  4.56316244e+04  1.40185392e+12  4.70658660e+17]
 [-1.81128023e-02  1.60643673e+04  1.53131889e+12  1.29652664e+18]]
--------------------
Theorical Mean [1.e-09 1.e-03 1.e+04 1.e+10]
Empirical mean [ 5.52872036e-04  1.10114984e+02 -6.73643338e+09  3.07299494e+15]
Mean diff      [-5.52871036e-04 -1.10113984e+02  6.73644338e+09 -3.07298494e+15]
--------------------
Theorical Std [1.e-01 1.e+05 1.e+12 1.e+18]
Empirical std [1.01075913e-01 9.98813245e+04 1.00412030e+12 1.00291456e+18]
Std diff      [-1.07591331e-03  1.18675504e+02 -4.12029854e+09 -2.91455683e+15]
--------------------
CNH s_abs:  [-6.74172926 -6.71596597 -6.79457752 -6.7734558 ]
CNH s_rel:  [-6.71082939 -6.70566